#### Joint task

* TMB： TMB >= 10 mut/Mb
* pTMB:  Tumors were classified into high or low groups for TMB and pTMB using the second tertile value.
* OS： OS >1 year (long OS) and patients with OS <1 year (short OS)
* MSI >= 10


In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)

tcga_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/'
tcga_patient = pd.read_pickle(os.path.join(tcga_data_path, 'PATIENTS.v2.TABLE'))

In [2]:
tcga_label = tcga_patient
def group_binary(df, tobin, q = 0.75):
    thrs = df.groupby('cancer_type').apply(lambda x: x[tobin].quantile(q))
    
    bins = df.groupby('cancer_type').apply(lambda x: (x[tobin] > x[tobin].quantile(q)))
    bins = bins.to_frame(name= tobin).reset_index().set_index('bcr_patient_barcode')
    return bins.loc[df.index][tobin], thrs

thr = int(tcga_label.age.quantile(0.75))
print(int(thr))
adp_age = (tcga_label.age >= 70).map({True:'age>%s' % thr, False:'age<=%s' % thr})
adp_gender = tcga_label.gender

adp_tumor_status = tcga_label.tumor_status.map({'TUMOR FREE':'Tumor_free' , 
                                                'WITH TUMOR':'With_tumor'})
adp_treatment_outcome = tcga_label.treatment_outcome.map({'R':'Response' , 
                                                'NR':'Nonresponse_Unknow'})
adp_treatment_outcome = adp_treatment_outcome.fillna('Nonresponse_Unknow')


tmb_thr = np.log2(10) #round(tcga_label.tmb.quantile(2/3),2) #
print(tmb_thr)
adp_tmb = (tcga_label.tmb >= tmb_thr).map({True:'High-TMB', False:'Low-TMB'})

ptmb_thr = round(tcga_label.Noushin_pTMB.quantile(4/5),2)
print(ptmb_thr)
adp_ptmb = (tcga_label.Noushin_pTMB >= ptmb_thr).map({True:'High-pTMB', False:'Low-pTMB'})

cnv_thr = round(tcga_label.cnv.quantile(4/5),2)
print(cnv_thr)
adp_cnv = (tcga_label.cnv >= cnv_thr).map({True:'High-CNV', False:'Low-CNV'})

adp_msi = tcga_label.msi.map({'MSI-H':'MSI-high','MSS':'MSS', 'MSI-L':'MSS'})

#os_thr = int(tcga_label.os_time.quantile(0.75))
os_thr = 10
print(os_thr)
adp_os_time = (tcga_label.os_time >= os_thr).map({True:'OS-long', False:'OS-short'})
adp_os_status = tcga_label.os_status

#pf_thr = int(tcga_label.pfi_time.quantile(0.75))
pf_thr = 10
print(pf_thr)
adp_pfi_time = (tcga_label.pfi_time >= pf_thr).map({True:'PFS-long', False:'PFS-short'})
adp_pfi_status = tcga_label.pfi_status

df_binary = pd.concat([tcga_label.cancer_type, adp_age, adp_gender, adp_tumor_status, 
                       adp_treatment_outcome, adp_tmb, adp_ptmb, 
                       adp_cnv, adp_msi, adp_os_time, adp_os_status, 
                       adp_pfi_time, adp_pfi_status], axis=1).loc[tcga_patient.index]

70
3.321928094887362
5.25
-0.05
10
10


In [3]:
s1 = []
p1 = []
for col in df_binary.columns[1:]:
    ss = df_binary[['cancer_type', col]].groupby('cancer_type')[col].apply(lambda x:x.value_counts()).unstack().T
    pp = (ss / ss.sum()).apply(lambda x:x.map("{:.2%}".format))
    s1.append(ss)
    p1.append(pp)
    
dfpp = pd.concat(p1)
dfss = pd.concat(s1)
dfpp.columns = dfpp.columns.map(lambda x:x.split('-')[1])
dfss.columns = dfss.columns.map(lambda x:x.split('-')[1])

dfsp = dfss.fillna(0).astype(int).astype(str) + "(" + dfpp + ")"#[['SKCM', 'KIRC', 'GBM','BLCA']]

dfsp[['SKCM', 'KIRC', 'GBM','BLCA']]

cancer_type,SKCM,KIRC,GBM,BLCA
age<=70,315(70.95%),385(74.61%),122(75.78%),210(53.03%)
age>70,129(29.05%),131(25.39%),39(24.22%),186(46.97%)
FEMALE,170(38.29%),185(35.85%),56(34.78%),106(26.77%)
MALE,274(61.71%),331(64.15%),105(65.22%),290(73.23%)
Tumor_free,209(48.05%),345(70.12%),9(6.21%),216(58.06%)
With_tumor,226(51.95%),147(29.88%),136(93.79%),156(41.94%)
Nonresponse_Unknow,444(100.00%),395(76.55%),160(99.38%),150(37.88%)
Response,0(nan%),121(23.45%),1(0.62%),246(62.12%)
High-TMB,164(36.94%),1(0.19%),3(1.86%),59(14.90%)
Low-TMB,280(63.06%),515(99.81%),158(98.14%),337(85.10%)


In [ ]:
#df_binary.to_pickle(os.path.join(fpath, 'PATIENT.PROCESSED.TCGA.TABLE'))